# ライブラリの読み込み

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
from collections import Counter
import ipadic
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE

import MeCab
import spacy

import re


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from scikitplot.metrics import plot_confusion_matrix, plot_roc

# データの読み込み

In [2]:
DATA_DIR = "../data/"

train_raw = pd.read_csv(f"{DATA_DIR}raw/train.csv")
train_raw.head(3)

,id,source,text,label
0,80074aa43,news4vip,まともに相手されてない人との関係なんて\nそんな大事にするものか？,0
1,6378fea6b,livejupiter,最近はアヘアヘQSマンやない？ ｲｲ!(・∀・)+1-0(・Ａ・)ｲｸﾅｲ!,0
2,c535f5613,livejupiter,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ\n甘えるな,1


In [3]:
print(pd.read_excel(f"{DATA_DIR}raw/data_explanation.xlsx", sheet_name=None).keys())
data_exp = pd.read_excel(f"{DATA_DIR}raw/data_explanation.xlsx", sheet_name="カラム説明")
data_exp

dict_keys(['ファイル説明', 'カラム説明'])


,カラム名,内容
0,id,一意のid
1,source,おーぷん2ちゃんねる対話コーパスでクロールされた掲示板(「なんでも実況(ジュピター)(liv...
2,text,投稿テキスト
3,label,0:ヘイトスピーチではない。1:ヘイトスピーチ


# trainデータ

In [4]:
train = train_raw.copy()

In [5]:
train.isnull().sum()

id        0
source    0
text      0
label     0
dtype: int64

In [6]:
train.describe(include="all")

,id,source,text,label
count,5256,5256,5256,5256.000000
unique,5256,3,5256,NaN
top,80074aa43,newsplus,まともに相手されてない人との関係なんて\nそんな大事にするものか？,NaN
freq,1,1967,1,NaN
mean,NaN,NaN,NaN,0.058219
std,NaN,NaN,NaN,0.234180
min,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,0.000000
50%,NaN,NaN,NaN,0.000000
75%,NaN,NaN,NaN,0.000000


# 改行文字等削除

In [7]:
new_text = []
char_num = []

for line in train["text"]:
    line = line.strip("\n")
    line = line.replace("\n", "")
    line = line.rstrip("\u3000")
    line = line.replace("\u3000", "")
    
    # 半角全角英数字除去
    line = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', " ", line)
    # 記号もろもろ除去
    line = re.sub(r'[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=)／*&^%$#@!~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+', "", line)
    
    new_text.append(line)
    char_num.append(len(line))

In [8]:
train["text"] = new_text
train["char_num"] = char_num

train.head(3)

,id,source,text,label,char_num
0,80074aa43,news4vip,まともに相手されてない人との関係なんてそんな大事にするものか,0,30
1,6378fea6b,livejupiter,最近はアヘアヘ マンやない ｲｲ∀ ｲｸﾅｲ,0,24
2,c535f5613,livejupiter,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ甘えるな,1,45


# 形態素解析（spaCy/GiNZA）

In [9]:
nlp = spacy.load("ja_ginza")

In [10]:
# 挙動確認（noun_chunks）
doc = nlp(train["text"][0])

print(doc)

for np in doc.noun_chunks:
    print(np)

まともに相手されてない人との関係なんてそんな大事にするものか
まともに相手されてない人
関係
そんな
大事にするもの


In [11]:
# 挙動確認（品詞分解等）
for token in doc:
    print("token[%2d] = %-10s, pos:%-6s, dep:%-10s, head=%d" % (token.i, token.text, token.pos_, token.dep_, token.head.i))

token[ 0] = まとも       , pos:ADJ   , dep:advcl     , head=2
token[ 1] = に         , pos:AUX   , dep:aux       , head=0
token[ 2] = 相手        , pos:NOUN  , dep:acl       , head=7
token[ 3] = さ         , pos:AUX   , dep:aux       , head=2
token[ 4] = れ         , pos:AUX   , dep:aux       , head=2
token[ 5] = て         , pos:AUX   , dep:aux       , head=2
token[ 6] = ない        , pos:AUX   , dep:aux       , head=2
token[ 7] = 人         , pos:NOUN  , dep:nmod      , head=10
token[ 8] = と         , pos:ADP   , dep:case      , head=7
token[ 9] = の         , pos:ADP   , dep:case      , head=7
token[10] = 関係        , pos:NOUN  , dep:obl       , head=15
token[11] = なんて       , pos:ADP   , dep:case      , head=10
token[12] = そんな       , pos:PRON  , dep:obl       , head=15
token[13] = 大事        , pos:ADJ   , dep:advcl     , head=15
token[14] = に         , pos:AUX   , dep:aux       , head=13
token[15] = する        , pos:VERB  , dep:acl       , head=16
token[16] = もの        , pos:NOUN  , dep:ROOT     

In [12]:
# 挙動確認（品詞の依存関係）
from spacy import displacy
displacy.render(doc, style="dep", options={"compact":True})

In [13]:
# 名詞を入れるリスト
all_noun = []
# 名詞数を入れるリスト
all_noun_count = []

for sentence in train["text"]:
    doc = nlp(sentence)

    # 一文の名詞数を入れるリスト
    noun_count = []

    for token in doc:
        if token.pos_ == "NOUN":
            noun_count.append(token.text)
    all_noun.append(noun_count)
    all_noun_count.append(len(noun_count))

In [14]:
train["text_noun"] = all_noun
train["noun_num"] = all_noun_count
train.head(3)

,id,source,text,label,char_num,text_noun,noun_num
0,80074aa43,news4vip,まともに相手されてない人との関係なんてそんな大事にするものか,0,30,"[相手, 人, 関係, もの]",4
1,6378fea6b,livejupiter,最近はアヘアヘ マンやない ｲｲ∀ ｲｸﾅｲ,0,24,"[最近, マン, ｲｸﾅｲ]",3
2,c535f5613,livejupiter,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ甘えるな,1,45,"[日本人, 低学歴, 日本人, 権利, 甘える]",5


# Tfidf実装

In [15]:
# 名詞、動詞、形容詞を入れるリスト
all_noun = []

for sentence in train["text"]:
    doc = nlp(sentence)

    noun_count = []

    for token in doc:
        if token.pos_ == "NOUN":
            noun_count.append(token.text)
        if token.pos_ == "VERB":
            noun_count.append(token.text)
        if token.pos_ == "ADJ":
            noun_count.append(token.text)
    all_noun.append(noun_count)

In [16]:
train["text_tfidf"] = all_noun
train.head(3)

,id,source,text,label,char_num,text_noun,noun_num,text_tfidf
0,80074aa43,news4vip,まともに相手されてない人との関係なんてそんな大事にするものか,0,30,"[相手, 人, 関係, もの]",4,"[まとも, 相手, 人, 関係, 大事, する, もの]"
1,6378fea6b,livejupiter,最近はアヘアヘ マンやない ｲｲ∀ ｲｸﾅｲ,0,24,"[最近, マン, ｲｸﾅｲ]",3,"[最近, アヘアヘ, マン, ない, ｲｲ, ｲｸﾅｲ]"
2,c535f5613,livejupiter,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ甘えるな,1,45,"[日本人, 低学歴, 日本人, 権利, 甘える]",5,"[日本人, 生まれ, 無能, 低学歴, 分かっ, 日本人, 権利, 剥奪, 追放, す, 甘える]"


In [17]:
train_test = []

for words in train["text_tfidf"]:
    train_test.append(" ".join(words))

train["text_tfidf"] = train_test

In [18]:
train.head(3)

,id,source,text,label,char_num,text_noun,noun_num,text_tfidf
0,80074aa43,news4vip,まともに相手されてない人との関係なんてそんな大事にするものか,0,30,"[相手, 人, 関係, もの]",4,まとも 相手 人 関係 大事 する もの
1,6378fea6b,livejupiter,最近はアヘアヘ マンやない ｲｲ∀ ｲｸﾅｲ,0,24,"[最近, マン, ｲｸﾅｲ]",3,最近 アヘアヘ マン ない ｲｲ ｲｸﾅｲ
2,c535f5613,livejupiter,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ甘えるな,1,45,"[日本人, 低学歴, 日本人, 権利, 甘える]",5,日本人 生まれ 無能 低学歴 分かっ 日本人 権利 剥奪 追放 す 甘える


# ベクトル化

In [19]:
X = train["text_tfidf"]
y = train['label'].values

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 42)

In [21]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)

In [22]:
X_train.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# ナイーブベイズ

In [23]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

X_test = vectorizer.transform(X_test)

In [24]:
print(clf.score(X_train, y_train)) 
print(clf.score(X_test, y_test)) 

0.9405328258801142
0.9467680608365019
